In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

import glob, os
glob.glob(os.path.join(os.environ["SPARK_HOME"], "conf", "spark*"))

['/Users/rushi/spark/conf/spark-env.sh.template',
 '/Users/rushi/spark/conf/spark-defaults.conf',
 '/Users/rushi/spark/conf/spark-defaults.conf.template']

In [2]:
#read csv

testdf = pd.read_csv('datasets/test.csv')
traindf = pd.read_csv('datasets/train.csv')
alldf = pd.read_csv('datasets/submit.csv')

print(testdf.shape, traindf.shape)

print('cleaning dataframes')

testdf = testdf.dropna()
traindf = traindf.dropna()

print(testdf.shape, traindf.shape)

ytestdf = testdf.drop(['title', 'author', 'text'], axis=1)

# ytestdf['labels'] = np.zeros((len(testdf), 1))

(5200, 4) (20800, 5)
cleaning dataframes
(4575, 4) (18285, 5)


In [3]:
# ytestdf = ytestdf.merge(alldf, left_on='id', right_on='id', how='left')[['id', 'label']]
ytestdf = pd.merge(left = ytestdf, right=alldf, left_on='id', right_on='id', how='left')

In [4]:
spark = SparkSession.builder.\
    master('local[*]').\
        config('spark.driver.memory', '10g').\
            appName('fakenews').\
                getOrCreate()

print(spark.sparkContext.uiWebUrl)
print(spark.sparkContext.getConf().getAll())
    # master('local[*]').\
    #     config('spark.driver.memory','10g').\
print(spark)

http://d-i89-236-227.student.eduroam.uq.edu.au:4040
[('spark.driver.host', 'd-i89-236-227.student.eduroam.uq.edu.au'), ('spark.app.name', 'fakenews'), ('spark.driver.port', '57794'), ('spark.sql.warehouse.dir', 'file:/Users/rushi/Documents/GitHub/fake-news-detection/spark-warehouse'), ('spark.driver.memory', '10g'), ('spark.executor.id', 'driver'), ('spark.app.startTime', '1648515368329'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.app.id', 'local-1648515369602'), ('spark.ui.showConsoleProgress', 'true')]


In [5]:
from pyspark.sql.types import StructField, StructType
from pyspark.sql.types import IntegerType, StringType, DoubleType

trainschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType()),\
                StructField('label', IntegerType())]) 

testschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType())]) 

trainset = spark.read.csv('datasets/train.csv', schema=trainschema).dropna()
testset = spark.read.csv('datasets/test.csv', schema=testschema).dropna()

print(trainset.count(), testset.count())

trainset.printSchema()
testset.printSchema()

11844 5072
root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- text: string (nullable = true)



In [6]:
trainset.show(10)

testset.show(10)

+---+--------------------+--------------------+--------------------+-----+
| id|               title|              author|                text|label|
+---+--------------------+--------------------+--------------------+-----+
|  1|FLYNN: Hillary Cl...|     Daniel J. Flynn|Ever get the feel...|    0|
|  5|Jackie Mason: Hol...|     Daniel Nussbaum|In these trying t...|    0|
|  7|Benoît Hamon Wins...|     Alissa J. Rubin|PARIS  —   France...|    0|
|  8|Excerpts From a D...|                 nan|Donald J. Trump i...|    0|
|  9|A Back-Channel Pl...|Megan Twohey and ...|A week before Mic...|    0|
| 10|Obama’s Organizin...|         Aaron Klein|Organizing for Ac...|    0|
| 11|"BBC Comedy Sketc...|     Chris Tomlinson|The BBC produced ...|    0|
| 15|In Major League S...|       Jack Williams|Guillermo Barros ...|    0|
| 16|Wells Fargo Chief...|Michael Corkery a...|The scandal engul...|    0|
| 19|Chuck Todd: ’Buzz...|           Jeff Poor|Wednesday after  ...|    0|
+---+--------------------

In [7]:
# xtrain, ytrain = trainset[['id', 'title', 'author', 'text']], trainset[['id', 'label']]

# xtrain.show(10)
# ytrain.show(10)

In [13]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, CountVectorizer, StopWordsRemover, IDF
#tonkenize text

tokenizer = Tokenizer(outputCol='words')
tokenizer.setInputCol('text')
wordsdata = tokenizer.transform(trainset)

stopwr = StopWordsRemover(inputCol='words', outputCol='filtered')
wordsdata = stopwr.transform(wordsdata)

hashing = HashingTF(inputCol='filtered', outputCol='featuresHTF')
wordsdata = hashing.transform(wordsdata)

hashingIDF = IDF(inputCol='featuresHTF', outputCol='featuresIDF')
model = hashingIDF.fit(wordsdata)
wordsdata = model.transform(wordsdata)

# vector = CountVectorizer(inputCol='filtered', outputCol='featuresCV')
# model = vector.fit(wordsdata)

# wordsdata = model.transform(wordsdata)


wordsdata.show(10)

+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
| id|               title|              author|                text|label|               words|            filtered|         featuresHTF|         featuresIDF|
+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|  1|FLYNN: Hillary Cl...|     Daniel J. Flynn|Ever get the feel...|    0|[ever, get, the, ...|[ever, get, feeli...|(262144,[796,917,...|(262144,[796,917,...|
|  5|Jackie Mason: Hol...|     Daniel Nussbaum|In these trying t...|    0|[in, these, tryin...|[trying, times,, ...|(262144,[1689,425...|(262144,[1689,425...|
|  7|Benoît Hamon Wins...|     Alissa J. Rubin|PARIS  —   France...|    0|[paris, , —, , , ...|[paris, , —, , , ...|(262144,[965,1576...|(262144,[965,1576...|
|  8|Excerpts From a D...|                 nan

In [14]:
#create a model
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol='featuresIDF', labelCol='label', maxIter=2)
gbt.fit(wordsdata)

Py4JJavaError: An error occurred while calling o581.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 25.0 failed 1 times, most recent failure: Lost task 9.0 in stage 25.0 (TID 195) (d-i89-236-227.student.eduroam.uq.edu.au executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2454)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2403)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2402)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2402)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTas

In [ ]:
spark.stop()